In [ ]:
from keras import models, layers
from keras.applications import inception_v3
from keras import backend as k

In [9]:
k.set_learning_phase(0)
model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

In [10]:
layer_contributions = {
    'mixed2': 0.2,
    'mixed3': 3.,
    'mixed4': 2.,
    'mixed5': 1.5,
}

In [11]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])
loss = k.variable(0.)
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output

    scaling = k.prod(k.cast(k.shape(activation), 'float32'))
    loss += coeff * k.sum(k.square(activation[:, 2:-2, 2:-2, :])) / scaling

In [26]:
activation[:,2:-2,2:-2,:]

<tf.Tensor 'strided_slice_5:0' shape=(?, ?, ?, 768) dtype=float32>

In [25]:
activation

<tf.Tensor 'mixed5/concat:0' shape=(?, ?, ?, 768) dtype=float32>

In [23]:
loss

<tf.Tensor 'add_3:0' shape=() dtype=float32>

In [24]:
scaling

<tf.Tensor 'Prod_3:0' shape=() dtype=float32>

In [13]:
v_1 = k.constant([1, 2, 3, 4], dtype='int32')
v_2 = k.constant([2, 1, 5, 3], dtype='int32')
v_3 = k.constant([[1, 2, 3], [4, 5, 6]], dtype='int32')

In [14]:
k.prod(v_3, axis=1)

<tf.Tensor 'Prod_4:0' shape=(2,) dtype=int32>

In [15]:
import numpy as np

In [ ]:
[
    [
        [2, 4, 6], [2, 4, 6]
    ], [
        [1, 1, 1], [2, 2, 2]
    ]
]

In [16]:
x = np.array([[[2, 4, 6], [2, 4, 6]], [[1, 1, 1], [2, 2, 2]]])
print('0 :\n', np.prod(x, axis=0))
print('1 :\n', np.prod(x, axis=1))
print('2 :\n', np.prod(x, axis=2))

0 :
 [[ 2  4  6]
 [ 4  8 12]]
1 :
 [[ 4 16 36]
 [ 2  2  2]]
2 :
 [[48 48]
 [ 1  8]]


In [17]:
np.sum(np.square(x))

127

In [18]:
layer_dict['mixed1'].output

<tf.Tensor 'mixed1/concat:0' shape=(?, ?, ?, 288) dtype=float32>

In [19]:
layer_dict['mixed1'].output[:, 2:-2, 2:-2, :]

<tf.Tensor 'strided_slice_4:0' shape=(?, ?, ?, 288) dtype=float32>

In [20]:
layer_dict['mixed2'].output

<tf.Tensor 'mixed2/concat:0' shape=(?, ?, ?, 288) dtype=float32>

In [21]:
k.shape(layer_dict['mixed2'].output)

<tf.Tensor 'Shape_4:0' shape=(4,) dtype=int32>

In [22]:
layer_dict

{'activation_1': <keras.layers.core.Activation at 0x1a6e9a90>,
 'activation_10': <keras.layers.core.Activation at 0x1c5a64e0>,
 'activation_11': <keras.layers.core.Activation at 0x1c68e4e0>,
 'activation_12': <keras.layers.core.Activation at 0x1c7d8f28>,
 'activation_13': <keras.layers.core.Activation at 0x1ccbcd68>,
 'activation_14': <keras.layers.core.Activation at 0x1d1bb7b8>,
 'activation_15': <keras.layers.core.Activation at 0x1d2a4780>,
 'activation_16': <keras.layers.core.Activation at 0x1d38c780>,
 'activation_17': <keras.layers.core.Activation at 0x1d476748>,
 'activation_18': <keras.layers.core.Activation at 0x1d563780>,
 'activation_19': <keras.layers.core.Activation at 0x1d6b19b0>,
 'activation_2': <keras.layers.core.Activation at 0x6d8cda0>,
 'activation_20': <keras.layers.core.Activation at 0x1d792cc0>,
 'activation_21': <keras.layers.core.Activation at 0x1d81ecf8>,
 'activation_22': <keras.layers.core.Activation at 0x1d907cc0>,
 'activation_23': <keras.layers.core.Activa

In [ ]:
dream = model.input

grads = k.gradients(loss, dream)[0]
grads /= k.maximum(k.mean(k.abs(grads)), 1e-7)

outputs = [loss, grads]
fetch_loss_and_grads = k.function([dream], outputs)

def eval_loss_and_grads(x):
    outs=fetch_loss_and_grads([x])
    loss_value=outs[0]
    grad_values=outs[1]
    return loss_value,grad_values

def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...loss value as', i, 'loss_value')
        x += step * grad_values
    return x

In [ ]:
import numpy as np
from keras.preprocessing import image

import scipy

In [ ]:
def resize_img(img, size):
    img = np.copy(img)
    factors = (1, float(size[0]) / img.shape[1], float(size[1]) / img.shape[2],
               1)
    return scipy.ndimage.zoom(img, factors, order=1)


def save_img(img, fname):
    pil_img = deprecess_image(np.copy(img))
    scipy.misc.imsave(fname, pil_img)


def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img


def deprocess_image(x):
    if k.image_data_format == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
        x /= 2.
        x += 0.5
        x *= 255.
        x = np.clip(x, 0, 255).astype('uint8')
        return x

In [ ]:
step = 0.01
num_octave = 3
iterations = 20

max_loss = 10
base_image_path = '...'
img = preprocess_image(base_image_path)

original_shape = img.shape[1:3]
successive_shapes[original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / octave_scale**i) for dim in original_shape])
    successive_shapes.append(shape)
    successive_shapes = successive_shapes[::1]

    original_img = np.copy(img)
    shrunk_original_img = resize_img(img, successive_shapes[0])

    for shape in successive_shapes:
        print('Processing image shape: ', shape)
        img = resize_img(img, shape)
        img = gradient_ascent(img,
                              iterations=iterations,
                              step=step,
                              max_loss=max_loss)

        upscaled_shrunk_original_img = resize_img(shrunk_original_img, shapee)
        same_size_original = resize_img(original_img, shape)
        lost_detail = same_size_original - upscaled_shrunk_original_img

        img += lost_detail
        shrunk_original_img = resize_img(original_img, shape)
        save_img(img, fname='dream_as_scale' + str(shape) + '.png')

    save_img(img, fname='final_dream.png')